- Update: 2023.03.13

# 0. 配置参数

- `menu`：目录文件的路径
- `out`：保存结果的路径

In [1]:
menu = '../../file/walmart-fram_cabin_air_filter-content80.xlsx'
# pg.position()
position_url = (200, 60)
position_html = (560, 166)

out = '../../file/walmart-fram_cabin_air_filter-contentWHAT_THE_FUCK.xlsx'

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel(menu, header=0).fillna('')

df_menu

,item,url,图片链接,标题,价格,品牌,型号,工厂,工厂号,原价
0,251YDTGS5NBT,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,https://i5.walmartimages.com/asr/c4085188-314f...,FRAM Fresh Breeze Cabin Air Filter CF12157 wit...,9.81,FRAM,CF12156,Fram,CF12156,
1,5G3J28NSU00N,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,https://i5.walmartimages.com/asr/c4085188-314f...,FRAM Fresh Breeze Cabin Air Filter CF9119A wit...,12.34,FRAM,CF9119A,Fram,CF9119A,
2,40O9CYMZ2HEE,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,https://i5.walmartimages.com/asr/c4085188-314f...,FRAM Fresh Breeze CF9846A Cabin Air Filter for...,12.89,FRAM,CF9846A,Fram,CF9846A,
3,2H57Y1WR511V,https://www.walmart.com/ip/FRAM-CF11777-Fresh-...,https://i5.walmartimages.com/asr/c4085188-314f...,"FRAM CF11777, Fresh Breeze Cabin Air Filter wi...",13.06,FRAM,CF11777,Fram,CF11777,
4,3V2XR43TD20A,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,https://i5.walmartimages.com/asr/c4085188-314f...,FRAM Fresh Breeze CF11809 Cabin Air Filter for...,13.41,FRAM,CF11809,Fram,CF11809,
...,...,...,...,...,...,...,...,...,...,...
100,0WRQLUP5URW8,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,,,,,,,,
101,6A1NT2VS32QX,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,,,,,,,,
102,1Y6P4BUQZBAX,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,,,,,,,,
103,1A9Q4C4QMNM0,https://www.walmart.com/ip/FRAM-Fresh-Breeze-C...,,,,,,,,


In [4]:
import pyautogui as pg
import time
import pyperclip as pc
from lxml import etree
from bs4 import BeautifulSoup

In [5]:
pg.PAUSE = 0.3
pg.FAILSAFE = True

In [6]:
def get_html(url):
    pg.moveTo(position_url)
    pg.click()
    pg.hotkey('ctrl', 'a')
    pg.press('delete')
    
    pc.copy(url)
    pg.hotkey('ctrl', 'v')
    pg.press('enter')
    time.sleep(10)
    
    pg.press('f12')
    time.sleep(1.5)
    
    pg.moveTo(position_html)
    pg.click()
    pg.hotkey('ctrl', 'c')
    
    pg.press('f12')
    
    return pc.paste()

def parse_html(html):
    selector = etree.HTML(pc.paste())
    
    if len(selector.xpath('//*[@data-testid="zoom-image"]/img')) != 0:
        href = BeautifulSoup(etree.tostring(selector.xpath('//*[@data-testid="zoom-image"]/img')[0]).decode()).find('img')['src']
    elif len(selector.xpath('//*[@class="mr3 ml7 self-center relative"]/div/img')) != 0:
        href = BeautifulSoup(etree.tostring(selector.xpath('//*[@class="mr3 ml7 self-center relative"]/div/img')[0]).decode()).find('img')['src']
    else:
        print('what the fuck')
    title = BeautifulSoup(etree.tostring(selector.xpath('//*[@class="b lh-copy dark-gray mt1 mb2 f3"]')[0]).decode()).text
    price = BeautifulSoup(etree.tostring(selector.xpath('//*[@class="inline-flex flex-column"]')[0]).decode()).text.replace('$', '')
    if len(selector.xpath('//*[@class="mr2 f6 gray strike"]')) != 0:
        original_price = BeautifulSoup(etree.tostring(selector.xpath('//*[@class="mr2 f6 gray strike"]')[0]).decode()).text.replace('$', '')
    else:
        original_price = ''
    list_key = selector.xpath('//*[@class="flex items-center mv0 lh-copy f5 pb1 dark-gray"]')
    list_key = [BeautifulSoup(etree.tostring(i).decode()).text for i in list_key]
    if '' in list_key:
        list_key.remove('')
    list_value = selector.xpath('//*[@class="mv0 lh-copy f6 mid-gray"]')
    list_value = [BeautifulSoup(etree.tostring(i).decode()).text for i in list_value]
    brand = list_value[list_key.index('Brand')] if 'Brand' in list_key else ''
    model = list_value[list_key.index('Model')] if 'Model' in list_key else ''
    manufacturer = list_value[list_key.index('Manufacturer')] if 'Manufacturer' in list_key else ''
    number = list_value[list_key.index('Manufacturer Part Number')] if 'Manufacturer Part Number' in list_key else ''
    
    return href, title, price, original_price, brand, model, manufacturer, number

In [7]:
for i in range(len(df_menu)):
    if df_menu.loc[i, '图片链接'] == '':
        time.sleep(1)
        try:
            html = get_html(df_menu.loc[i, 'url'])
            print(i)
            time.sleep(1)
            df_menu.loc[i, '图片链接'], df_menu.loc[i, '标题'], df_menu.loc[i, '价格'], df_menu.loc[i, '原价'], df_menu.loc[i, '品牌'], df_menu.loc[i, '型号'], df_menu.loc[i, '工厂'], df_menu.loc[i, '工厂号'] = parse_html(html)
        finally:
            df_menu.to_excel(out.replace('WHAT_THE_FUCK', str(i+1)), index=False)
    else:
        print(f'第{i}行之前就搞定了啊哈哈哈哈哈！')

df_menu.to_excel(out.replace('WHAT_THE_FUCK', ''), index=False)
print('老铁666，全部搞定！！！')

第0行之前就搞定了啊哈哈哈哈哈！
第1行之前就搞定了啊哈哈哈哈哈！
第2行之前就搞定了啊哈哈哈哈哈！
第3行之前就搞定了啊哈哈哈哈哈！
第4行之前就搞定了啊哈哈哈哈哈！
第5行之前就搞定了啊哈哈哈哈哈！
第6行之前就搞定了啊哈哈哈哈哈！
第7行之前就搞定了啊哈哈哈哈哈！
第8行之前就搞定了啊哈哈哈哈哈！
第9行之前就搞定了啊哈哈哈哈哈！
第10行之前就搞定了啊哈哈哈哈哈！
第11行之前就搞定了啊哈哈哈哈哈！
第12行之前就搞定了啊哈哈哈哈哈！
第13行之前就搞定了啊哈哈哈哈哈！
第14行之前就搞定了啊哈哈哈哈哈！
第15行之前就搞定了啊哈哈哈哈哈！
第16行之前就搞定了啊哈哈哈哈哈！
第17行之前就搞定了啊哈哈哈哈哈！
第18行之前就搞定了啊哈哈哈哈哈！
第19行之前就搞定了啊哈哈哈哈哈！
第20行之前就搞定了啊哈哈哈哈哈！
第21行之前就搞定了啊哈哈哈哈哈！
第22行之前就搞定了啊哈哈哈哈哈！
第23行之前就搞定了啊哈哈哈哈哈！
第24行之前就搞定了啊哈哈哈哈哈！
第25行之前就搞定了啊哈哈哈哈哈！
第26行之前就搞定了啊哈哈哈哈哈！
第27行之前就搞定了啊哈哈哈哈哈！
第28行之前就搞定了啊哈哈哈哈哈！
第29行之前就搞定了啊哈哈哈哈哈！
第30行之前就搞定了啊哈哈哈哈哈！
第31行之前就搞定了啊哈哈哈哈哈！
第32行之前就搞定了啊哈哈哈哈哈！
第33行之前就搞定了啊哈哈哈哈哈！
第34行之前就搞定了啊哈哈哈哈哈！
第35行之前就搞定了啊哈哈哈哈哈！
第36行之前就搞定了啊哈哈哈哈哈！
第37行之前就搞定了啊哈哈哈哈哈！
第38行之前就搞定了啊哈哈哈哈哈！
第39行之前就搞定了啊哈哈哈哈哈！
第40行之前就搞定了啊哈哈哈哈哈！
第41行之前就搞定了啊哈哈哈哈哈！
第42行之前就搞定了啊哈哈哈哈哈！
第43行之前就搞定了啊哈哈哈哈哈！
第44行之前就搞定了啊哈哈哈哈哈！
第45行之前就搞定了啊哈哈哈哈哈！
第46行之前就搞定了啊哈哈哈哈哈！
第47行之前就搞定了啊哈哈哈哈哈！
第48行之前就搞定了啊哈哈哈哈哈！
第49行之前就搞定了啊哈哈哈哈哈！
第50行之前就搞定了啊哈哈哈哈哈！
第51行之前就搞定了啊哈哈哈哈哈！
第52行之前就搞定了啊哈哈哈哈哈！
第53行之前就搞定了啊哈哈哈哈哈！
第54行之前就搞定了啊哈哈哈哈哈！
第55行之前就搞定了啊哈哈哈哈哈！
第5